In [1]:
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.neighbors import *
import pandas as pd
import numpy as np

加载数据

In [2]:
table_head = ['anime_id', 'name', 'other_name', 'rank', 'rating']
an_list = pd.read_csv('animelist.csv', names=table_head)
an_list.shape

(3541, 5)

In [3]:
cm_list = pd.read_csv('comments.csv', low_memory=False)

In [4]:
cm_list

,user_id,user_name,anime_id,anime_name,user_rating,comment
0,100004,从不卖萌的K,8,Code Geass 反叛的鲁路修R2,8.0,朱雀洗白了还是黑 我不是车夫党
1,100004,从不卖萌的K,51,CLANNAD,10.0,神作 不解释 after story的铺垫
2,100004,从不卖萌的K,265,新世纪福音战士,10.0,神作不解释 坐等Q无压力
3,100004,从不卖萌的K,309,真实之泪,8.0,我是养女党
4,100004,从不卖萌的K,467,地球防卫少年,9.0,阿姨说大爱 好作品 看了就知道
...,...,...,...,...,...,...
832198,NaN,NaN,1741,黑塔利亚 第2期,NaN,= =好吧..作为男性我自重..
832199,NaN,只有沈知道的世界,2082,甜蜜声优,NaN,建议看mkv版。848x480宽屏格式。JS有载点。隐藏(？)ED推荐
832200,NaN,NaN,6811,海月姬,NaN,一个触手控和一个伪娘为了保护爱巢（误？）而成为时装设计师的故事= =？！
832201,NaN,NaN,9717,魔法少女小圆,NaN,这片叫 魔法少女不是小圆


In [4]:
cm_list = cm_list.dropna(subset=["user_id", "user_rating"])
cm_list.shape

(752762, 6)

把NaN值清洗掉，并提取出需要分析的列

In [5]:
cm_imp = cm_list[['user_id', 'anime_id', 'user_rating']]
cm_imp = cm_imp.merge(an_list[['anime_id', 'rating', 'name']], how="inner", left_on="anime_id", right_on="anime_id")
cm_imp

,user_id,anime_id,user_rating,rating,name
0,100004,8,8.0,8.2,Code Geass 反叛的鲁路修R2
1,101088,8,10.0,8.2,Code Geass 反叛的鲁路修R2
2,101365,8,7.0,8.2,Code Geass 反叛的鲁路修R2
3,101494,8,8.0,8.2,Code Geass 反叛的鲁路修R2
4,10341,8,8.0,8.2,Code Geass 反叛的鲁路修R2
...,...,...,...,...,...
752757,suidoreen,38758,7.0,7.0,闪电十一人GO 时空之石
752758,treesty,38758,5.0,7.0,闪电十一人GO 时空之石
752759,usagirenka,38758,8.0,7.0,闪电十一人GO 时空之石
752760,xelloss646,38758,7.0,7.0,闪电十一人GO 时空之石


把评分小于5的动画淘汰

In [6]:
bs = cm_imp['rating'] >= 5
cm_imp = cm_imp[bs]

In [8]:
an_list = an_list[an_list['rating'] >= 5]
an_list

,anime_id,name,other_name,rank,rating
0,253,星际牛仔,カウボーイビバップ,1,9.1
1,326,攻壳机动队 S.A.C. 2nd GIG,攻殻機動隊 S.A.C. 2nd GIG,2,9.1
2,324,攻壳机动队 STAND ALONE COMPLEX,攻殻機動隊 STAND ALONE COMPLEX,3,9.1
3,876,CLANNAD ～AFTER STORY～,NaN,4,9.0
4,25961,猫和老鼠,Tom and Jerry,7,8.9
...,...,...,...,...,...
3524,107070,环太平洋：黑色禁区,Pacific Rim: The Black,6330,5.2
3525,6934,星座彼氏,Starry☆Sky,6339,5.5
3526,322900,终末的女武神,終末のワルキューレ,6390,5.3
3527,315898,生化危机：无尽黑暗,BIOHAZARD：Infinite Darkness,6481,5.0


把评分人数小于100（冷门动画），评分个数小于50的用户（不活跃用户）淘汰

In [9]:
count1 = cm_imp['anime_id'].value_counts()
count2 = cm_imp['user_id'].value_counts()

In [10]:
bs1 = count1[count1 >= 100]
bs2 = count2[count2 >= 50]
rating_data = cm_imp[cm_imp['anime_id'].isin(bs1.index)].copy()
rating_data = rating_data[rating_data['user_id'].isin(bs2.index)].copy()
rating_data.head()

,user_id,anime_id,user_rating,rating,name
1,101088,8,10.0,8.2,Code Geass 反叛的鲁路修R2
3,101494,8,8.0,8.2,Code Geass 反叛的鲁路修R2
5,10348,8,9.0,8.2,Code Geass 反叛的鲁路修R2
6,104081,8,9.0,8.2,Code Geass 反叛的鲁路修R2
8,106930,8,8.0,8.2,Code Geass 反叛的鲁路修R2


In [11]:
an_list = an_list[an_list['anime_id'].isin(bs1.index)]
an_list

,anime_id,name,other_name,rank,rating
0,253,星际牛仔,カウボーイビバップ,1,9.1
1,326,攻壳机动队 S.A.C. 2nd GIG,攻殻機動隊 S.A.C. 2nd GIG,2,9.1
2,324,攻壳机动队 STAND ALONE COMPLEX,攻殻機動隊 STAND ALONE COMPLEX,3,9.1
3,876,CLANNAD ～AFTER STORY～,NaN,4,9.0
4,25961,猫和老鼠,Tom and Jerry,7,8.9
...,...,...,...,...,...
3514,234195,一脸嫌弃表情的妹子给你看胖次,嫌な顔されながらおパンツ見せてもらいたい,6124,5.7
3519,317693,天空侵犯,天空侵犯,6218,5.6
3525,6934,星座彼氏,Starry☆Sky,6339,5.5
3526,322900,终末的女武神,終末のワルキューレ,6390,5.3


把用户对动画大于等于7.0的评分标记为喜欢

In [12]:
# rating_data['like'] = (rating_data['user_rating'] >= 7.0).astype('int')
rating_data['like'] = rating_data['user_rating'] >= 7.0
rating_data.shape

(436689, 6)

In [14]:
an_list.sort_values('rank').to_csv('anime_list.csv', index=False)
rating_data.to_csv('rating_data.csv', index=False)

ItemKNN模式（计算两个动画间的距离）

In [41]:
like_table = rating_data.pivot_table(index="anime_id", columns="user_id", values="like", fill_value=0)

In [42]:
like_table_matrix = csr_matrix(like_table.values)
item_knn = NearestNeighbors(metric="cosine", algorithm="brute")
item_knn.fit(like_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

UserKNN模式（计算两个用户间的距离）

In [18]:
like_table2 = rating_data.pivot_table(index="user_id", columns="name", values="like", fill_value=0)
like_table_matrix2 = csr_matrix(like_table2.values)

name,11eyes,18：与你相连的拼图游戏,22/7,30岁的保健体育,3月的狮子,3月的狮子 第二季,86 -不存在的战区-,86 -不存在的战区- 第二季,91天,A.I.C.O.：化身,...,龙与虎,龙之界点！,龙娘七七七埋藏的宝藏,龙族买房,龙王的工作！,龙珠,龙珠GT,龙珠Z,龙珠超,龙的牙医
user_id,,,,,,,,,,,,,,,,,,,,,
101088,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
101412,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
101494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10172,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10183,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ztw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zty61211,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
zxsw,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
user_knn = NearestNeighbors(metric="cosine", algorithm="brute")
user_knn.fit(like_table_matrix2)

NearestNeighbors(algorithm='brute', metric='jaccard')

In [ ]:
id2name = {}
name2id = {}
for i in range(len(an_list)):
    aid = an_list.iloc[i]['anime_id']
    name = an_list.iloc[i]['name']
    id2name[aid] = name
    name2id[name] = aid

In [ ]:
def __predict(name, aid):
    if not aid:
        print("Cannot find {}".format(name))
        return
    query = like_table.loc[aid].values.reshape(1, -1)
    distance, suggestions = item_knn.kneighbors(query, n_neighbors=7)

    # ret = list(zip(suggestions.flatten(), distance.flatten()))
    # return ret
    ret = []

    for i in range(0, len(distance.flatten())):
        if i == 0:
            # print('Recommendations for {0}:\n'.format(name))
            continue
        else:
            idx = suggestions.flatten()[i]
            # print('{0}: {1} with distance of {2}:'.format(i, find_name_by_id(like_table.index[idx]), distance.flatten()[i]))
            ret.append((id2name[like_table.index[idx]], distance.flatten()[i]))
    return ret
            
            
def predict_by_id(aid):
    name = id2name[aid]
    return __predict(name, aid)


def predict_by_name(name):
    aid = name2id[name]
    return __predict(name, aid)

In [48]:
def predict_by_anime(anl):
    cnt = Counter()
    for my_an in anl:
        rec = predict_by_name(my_an)
        for re_an in rec:
            ani = re_an[0]
            if ani not in anl:
                cnt[ani] += 1
    return cnt


In [49]:
def predict_by_user(an_set):
    qe = pd.Series(0, index=like_table2.columns, dtype='int')
    for k in al:
        aid = name2id[k]
        qe[aid] = 1
    dist, sug = user_knn.kneighbors(qe.values.reshape(1, -1), n_neighbors=7)
    cnt = Counter()
    for i in range(len(dist.flatten())):
        user_like = like_table2.iloc[sug.flatten()[i]]
        for item in user_like.index:
            if user_like[item] == 1:
                if id2name[item] not in al:
                    cnt[id2name[item]] += 1
    return cnt

In [ ]:
al = set()
with open('示例1.txt', 'r', encoding='utf8') as f:
    for row in f.readlines():
        al.add(row.strip())

In [51]:
u_cnt = predict_by_user(al)
for k in u_cnt.most_common(20):
    print(k[0])

小林家的龙女仆
人渣的本愿
徒然Children
Re：从零开始的异世界生活
笨女孩
日在校园
NO GAME NO LIFE 游戏人生
ReLIFE
Angel Beats!
我们仍未知道那天所看见的花的名字。
Little Busters!
悠哉日常大王
山田君与7个魔女
夏洛特
紫罗兰永恒花园
狂赌之渊
龙与虎
魔法禁书目录
亲吻姐姐
我的妹妹哪有这么可爱！


d:\anaconda\lib\site-packages\sklearn\metrics\pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [54]:
a_cnt = predict_by_anime(al)
for k in a_cnt.most_common(20):
    print(k[0])

如果有妹妹就好了。
比宇宙更远的地方
末日时在做什么？有没有空？可以来拯救吗？
NO GAME NO LIFE 游戏人生
四月是你的谎言
我的青春恋爱物语果然有问题
欢迎来到实力至上主义教室
GAMERS电玩咖
辉夜大小姐想让我告白？～天才们的恋爱头脑战～
佐贺偶像是传奇
鬼灭之刃
约定的梦幻岛
ID:INVADED
我的脑内选项正在全力妨碍学园恋爱喜剧
安达与岛村
堀与宫村
魔女之旅
珈百璃的堕落
小林家的龙女仆
川柳少女
